In [13]:
import requests
from bs4 import BeautifulSoup
import re
import json

In [14]:
# yahoo 電影的網址後面都是那一部電影的電影ID，所以分析後來後，能方便使用
# https://movies.yahoo.com.tw/movieinfo_main/%E7%84%A1%E5%90%8D%E5%BC%92-nobody-11126
# 無名弒 的電影 ID 就是 11126

def get_movie_id(url):
    try:
        movie_id = url.split('.html')[0].split('-')[-1]
    except:
        movie_id = url
    return movie_id

In [15]:
url = 'https://movies.yahoo.com.tw/movieinfo_main/%E7%84%A1%E5%90%8D%E5%BC%92-nobody-11126'
print(get_movie_id(url))

11126


In [16]:
# '\d+' 表示一個或多個數字
def get_date(date_str):
    pattern = '\d+-\d+-\d+'
    match = re.search(pattern, date_str)
    if match is None:
        return date_str
    else:
        return match.group(0)

In [17]:
print(get_date('02-04-2021'))

02-04-2021


In [18]:
def get_movies(dom):
    soup = BeautifulSoup(dom,'html5lib')
    movies = []
    rows = soup.find_all('div','release_info_text')
    print(rows)
    for row in rows:
        movie = dict()
        movie['expectation'] = row.find('div','leveltext').span.text.strip()
        movie['ch_name'] = row.find('div','release_movie_name').a.text.strip()
        movie['eng_name'] = row.find('div','release_movie_name').find('div','en').a.text.strip()
        movie['movie_id'] = get_movie_id(row.find('div','release_movie_name').a['href'])
        movie['poster_url'] = row.parent.find_previous_sibling('div','release_foto').a.img['src']
        movie['release_date'] = get_date(row.find('div','release_movie_time').text)
        movie['intro']= row.find('div','release_text').text.replace(u'詳全文','').strip()
        trailer_a = row.find_next_sibling('div','release_btn color_btnbox').find_all('a')[1]
        movie['trailer_url'] = trailer_a['href'] if 'href' in trailer_a.attrs else ''
        movies.append(movie)
               
    return movies 

In [19]:
def get_web_page(url):
    resp = requests.get(
    url = url
    )
    if resp.status_code != 200:
        print('Invalid url:',resp.url)
    else:
        return resp.text



In [20]:
Y_MOVIE_URL = 'https://movies.yahoo.com.tw/movie_thisweek.html'

page = get_web_page(Y_MOVIE_URL)
if page:
    movies = get_movies(page)
    for movie in movies:
        print(movie)
    with open('movie.json','w',encoding = 'utf-8') as f:
        json.dump(movies,f,indent = 2 , sort_keys = True , ensure_ascii = False)
    

[<div class="release_info_text">
              <div class="release_movie_name">
                <a class="gabtn" data-ga="['本週新片','本週新片_本週新片第1頁','無名弒']" href="https://movies.yahoo.com.tw/movieinfo_main/%E7%84%A1%E5%90%8D%E5%BC%92-nobody-11126">
                  無名弒</a>
                <div class="en">
                  <a class="gabtn" data-ga="['本週新片','本週新片_本週新片第1頁','無名弒']" href="https://movies.yahoo.com.tw/movieinfo_main/%E7%84%A1%E5%90%8D%E5%BC%92-nobody-11126">
                    Nobody</a>
                </div>
                <dl class="levelbox">
                                    <dt>
                  <div class="level_name">期待度</div>
                  <div class="leveltext">
                    <span>81%</span>
                    網友想看
                  </div>
                  </dt>
                                                      <dd>
                    <div class="level_name">滿意度</div>
                    <div class="leveltext">
                      <span class=